In [4]:
import os
from typing import List
from pydantic import BaseModel
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langgraph.graph import StateGraph,END

In [2]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [3]:
llm=init_chat_model(model='groq:gemma2-9b-it')

In [5]:
docs=TextLoader('research_notes.txt').load()
chunks=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50).split_documents(docs)
vector_store=FAISS.from_documents(chunks,OpenAIEmbeddings(model='text-embedding-3-small'))
retriever=vector_store.as_retriever()

In [14]:
class RAGReflectionState(BaseModel):
    question:str
    retrieved_docs:List[Document]=[]
    answer:str=""
    reflection:str=""
    revised:bool=False
    attempts:int=0


In [15]:
def retrieve_docs(state:RAGReflectionState) -> RAGReflectionState:
    docs=retriever.invoke(state.question)
    return state.model_copy(update={'retrieved_docs':docs})

def generate_answer(state:RAGReflectionState) -> RAGReflectionState:
    context="\n\n".join([doc.page_content for doc in state.retrieved_docs])
    prompt=f"""Use the following context to answer the question:
    Context:{context}\n\n Question:{state.question}"""

    answer=llm.invoke(prompt).content.strip()
    return state.model_copy(update={'answer':answer,'attempts':state.attempts+1})

In [16]:
def reflect_on_answer(state:RAGReflectionState) -> RAGReflectionState:
    prompt=f"""Reflect on the following answer to see if it fully addresses the question.
    State YES if it complete and correct,or NO with an explanation.
    Question:{state.question}\n\n Answer:{state.answer}"
    
    Respond like:
    Reflection:YES or NO
    Explanation:...
    """

    result=llm.invoke(prompt).content
    is_ok="reflection:yes" in result.lower()
    return state.model_copy(update={'reflection':result,'revised':not is_ok})

In [17]:
def finalize(state:RAGReflectionState)->RAGReflectionState:
    return state

In [20]:
builder=StateGraph(RAGReflectionState)

builder.add_node('retriever',retrieve_docs)
builder.add_node('responder',generate_answer)
builder.add_node('reflector',reflect_on_answer)
builder.add_node('done',finalize)

builder.set_entry_point('retriever')
builder.add_edge('retriever','responder')
builder.add_edge('responder','reflector')
builder.add_conditional_edges(
    'reflector',
    lambda s:'done' if not s.revised or s.attempts >= 2 else 'retriever'

)
builder.add_edge('done',END)
graph=builder.compile()

In [21]:
if __name__=="__main__":
    query="How basically attention mechansim changed the overview of recurrent models? "
    init_state=RAGReflectionState(question=query)
    result=graph.invoke(init_state)

    print("\n Final Answer:\n",result['answer'])
    print("\n Reflection Log:\n",result['reflection'])
    print('Total Attempts:',result['attempts'])



 Final Answer:
 The attention mechanism fundamentally changed the landscape of recurrent models by offering a more efficient and effective way to process sequential data.  

Here's how:

* **Addressing Long-Range Dependencies:** Recurrent models like RNNs, LSTMs, and GRUs struggled to capture relationships between words that were far apart in a sequence due to the sequential nature of their processing. Attention, on the other hand, allows the model to weigh the importance of different words in a sequence, regardless of their position, enabling it to understand long-range dependencies better.

* **Parallelization:**  Transformers, which rely on attention, can process entire sequences in parallel, unlike RNNs which process information sequentially. This parallelization significantly speeds up training and makes it possible to work with much larger datasets.

* **Conceptual Shift:** Attention marked a paradigm shift away from solely relying on recurrence. It introduced a new way of think